<a href="https://colab.research.google.com/github/PhuongNhi187/TieuLuan_NguyenThiPhuongNhi_2024_2025/blob/main/run.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Chuẩn bị

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd '/content/gdrive/MyDrive/Tieu_luan/'

Mounted at /content/gdrive
/content/gdrive/MyDrive/Tieu_luan


In [ ]:
!pip install pyvi
!pip install python-multipart
!pip install fastapi uvicorn pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 37.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.4/341.4 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import re
from pyvi import ViTokenizer
import pandas as pd
import tensorflow as tf
import numpy as np
import joblib

In [ ]:
correct_char_path = pd.read_table('/content/gdrive/MyDrive/Tieu_luan/Pre-processing/correct_char.txt', delimiter='\t')
correct_char_dict = dict(zip(correct_char_path['char'], correct_char_path['correct_char']))

en2vi_path = pd.read_table('/content/gdrive/MyDrive/Tieu_luan/Pre-processing/top_500_adjective_eng.txt', delimiter='\t')
en2vi_dict = dict(zip(en2vi_path['Adj'], en2vi_path['Mean']))

teencode_path = pd.read_table('/content/gdrive/MyDrive/Tieu_luan/Pre-processing/teencode.txt', delimiter='\t')
teencode_dict = dict(zip(teencode_path['incorrect'], teencode_path['correct']))
def preprocess_text(text):
    # Bước 1
    text = text.lower()

    # Bước 2
    def remove_special_words(text):
        if not isinstance(text, str): return ""
        text = re.sub(r'\#\w+', '', text)   # Loại bỏ các từ có hashtag (#)
        text = re.sub(r'<.*?>', '', text)   # Loại bỏ các thẻ HTML (< >)
        text = re.sub(r'http\S+', '', text)   # Loại bỏ các URL (http://)
        text = re.sub(r'@\w+', '', text)    # Loại bỏ các mention (@)
        return text

    text = remove_special_words(text)

   # Bước 3
    def remove_special_characters(text):
        text_pre = re.compile("[^a-zA-Zàáảãạăắằẳẵặâấầẩẫậèéẻẽẹêếềểễệđíìỉĩịòóỏõọôốồổỗộơớờởỡợùúủũụưứừửữựỳýỷỹỵđ\s]")
        return re.sub(text_pre, ' ', text)

    def correct_char(text):
        words = text.split()
        for word in words:
            for k, v in correct_char_dict.items():
                if word == k:
                    key = ' '+k+' '
                    value = ' '+v+' '
                    text = text.replace(key, value)
        return text

    text = remove_special_characters(text)
    text = correct_char(text)


    # Bước 4
    def remove_repeated_characters(text):
        return re.sub(r'(\w)\1+', r'\1', text)

    def remove_repeated_words(text):
        words = text.split()
        seen = set()
        new_words = []
        for word in words:
            if word not in seen:
                seen.add(word)
                new_words.append(word)
        return " ".join(new_words)

    text = remove_repeated_characters(text)
    text = remove_repeated_words(text)

    # Bước 5
    def remove_emoji(text):
        emoji_pattern = re.compile("["
                                   u"\U0001F600-\U0001F64F"
                                   u"\U0001F300-\U0001F5FF"
                                   u"\U0001F680-\U0001F6FF"
                                   u"\U0001F1E0-\U0001F1FF"
                                   u"\U00002500-\U00002BEF"
                                   u"\U00002702-\U000027B0"
                                   u"\U00002702-\U000027B0"
                                   u"\U000024C2-\U0001F251"
                                   u"\U0001f926-\U0001f937"
                                   u"\U00010000-\U0010ffff"
                                   u"\u2640-\u2642"
                                   u"\u2600-\u2B55"
                                   u"\u200d"
                                   u"\u23cf"
                                   u"\u23e9"
                                   u"\u231a"
                                   u"\ufe0f"
                                   u"\u3030"
                                   "]+", flags=re.UNICODE)
        return emoji_pattern.sub(r' ', text)

    text = remove_emoji(text)

    # Bước 6
    def change_en_to_vn(text):

        words = text.split()
        for word in words:
            for k, v in en2vi_dict.items():
                if word == k:
                    key = ' '+k+' '
                    value = ' '+v+' '
                    text = text.replace(key, value)
        return text

    text = change_en_to_vn(text)

    # Bước 7
    def teencode(text):

        words = text.split()
        for word in words:
            for k, v in teencode_dict.items():
                if word == k:
                    key = ' '+k+' '
                    value = ' '+v+' '
                    text = text.replace(key, value)
        return text

    text = teencode(text)

    # Bước 8
    def remove_extra_spaces(text):
        return ' '.join(text.split())

    text = remove_extra_spaces(text)

    def remove_text_over_len(text):
        text = ' '.join([w for w in text.split() if len(w) <= 7])
        return text

    text = remove_text_over_len(text)

    # Tokenize tiếng Việt
    text = ViTokenizer.tokenize(text)

    return text

#Load model

In [ ]:
loaded_model = joblib.load('/content/gdrive/MyDrive/Tieu_luan/model_BiLSTM_TextVectorization.pkl')

#Xây dựng công cụ

In [ ]:
from fastapi import FastAPI, Form
from fastapi.responses import HTMLResponse
from threading import Thread
import uvicorn
from pyngrok import ngrok
import numpy as np
import joblib

# Khởi tạo ứng dụng FastAPI
app = FastAPI()

# Định nghĩa route (đường dẫn) cho form
@app.get("/", response_class=HTMLResponse)
async def home():
    return """
    <html>
        <head>
            <title>Đánh giá cảm xúc văn bản</title>
            <style>
             .center {
                      position: absolute;
                      top: 50%;
                      left: 50%;
                      transform: translate(-50%, -50%);
                  }
              #submitForm{
                      width: 400px;
                      padding: 20px;
                      border: 2px solid #ccc;
                      border-radius: 10px;
                      box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
                      background-color: #fff;
                      margin: auto;
                  }

                  label {
                      display: block;
                      margin-bottom: 10px;
                      font-size: 18px;
                  }

                  input[type="text"] {
                      width: 100%;
                      padding: 10px;
                      border: 1px solid #ccc;
                      border-radius: 5px;
                      font-size: 16px;
                  }

                  button {
                      margin-top: 20px;
                      background-color: #4caf50;
                      color: white;
                      padding: 10px 0;
                      border: none;
                      border-radius: 5px;
                      cursor: pointer;
                      font-size: 16px;
                      width: 100%;
                  }

                  button:hover {
                      background-color: #45a049;
                  }

                  #result {
                      margin-top: 20px;
                  }
            </style>
        </head>
        <body>
        <div class = "center">
            <form id="submitForm">
                <label for="text_input">Hãy nhập văn bản cần kiểm tra:</label><br>
                <input type="text" id="text_input" name="text_input" required><br>
                <button type="button" onclick="predict()">Kiểm tra</button>
                <div id="result"></div>
            </form>

        </div>
            <script>
                async function predict() {
                    const text = document.getElementById("text_input").value; // Lấy giá trị nhập vào từ thẻ input

                    // Gửi yêu cầu POST đến endpoint /predict
                    const response = await fetch('/predict', {
                        method: 'POST',
                        headers: {
                            'Content-Type': 'application/x-www-form-urlencoded',
                        },
                        body: new URLSearchParams({ text_input: text }),
                    });

                    // Đọc nội dung của phản hồi
                    const result = await response.text();

                    // Hiển thị kết quả dự đoán trong thẻ div có id là "result"
                    document.getElementById("result").innerHTML = result;
                }
            </script>
        </body>
    </html>
    """

@app.post("/predict", response_class=HTMLResponse)
async def predict(text_input: str = Form(...)):
    # Thực hiện dự đoán ở đây
    print(f"Received input: {text_input}")  # Thêm dòng log này để ghi lại dữ liệu text_input

    token_input = [preprocess_text(text_input)]
    token_input = np.array(token_input)
    prediction = loaded_model.predict(token_input)

    # Xác định kết quả và trả về
    if prediction[0][0] <= prediction[0][1]:
        result = "Chuỗi tích cực."
    else:
        result = "Chuỗi tiêu cực."

    html_response = f"<p>Chuỗi sau khi xử lí: {token_input}</p>"
    html_response += f"<p>Chỉ số tích cực: {prediction[0][1] *100:.2f}%</p>"
    html_response += f"<p>Chỉ số tiêu cực: {prediction[0][0] *100:.2f}%</p>"
    html_response += f"<p>Kết quả dự đoán: {result}</p>"
    return html_response

def run_app():
    uvicorn.run(app, host="0.0.0.0", port=4040)

thread = Thread(target=run_app)
thread.start()

ngrok.set_auth_token("2hhcjfMgajF0GkRezv7UFvg0b3D_4TxG17dk5izc1gEP5roHf")

ngrok_tunnel = ngrok.connect(4040)
print("Public URL:", ngrok_tunnel.public_url)


INFO:     Started server process [642]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:4040 (Press CTRL+C to quit)


Public URL: https://05f6-34-125-200-19.ngrok-free.app


In [ ]:
sentence = "sản phẩm không tốt"
text = preprocess_text(sentence)
print(text)
token_input = [text]
token_input = np.array(token_input)
prediction = loaded_model.predict(token_input)
print(prediction)

# lớp 0 là tiêu cực, 1 là tích cực

if prediction[0][0] <= prediction[0][1]:
    print("Chuỗi tích cực.")
else:
    print("Chuỗi tiêu cực.")


sản_phẩm không tốt
1/1 [==============================] - 0s 123ms/step
[[0.7297815  0.27025032]]
Chuỗi tiêu cực.
